<a href="https://colab.research.google.com/github/umarmaul/Recommendation_System/blob/main/UI_Incubate_Content_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Recomendation based user log

In [62]:
import numpy as np
import pandas as pd

In [63]:
df = pd.read_csv('https://raw.githubusercontent.com/althafnafi/incubate-app/main/dataset/dataset_makanan100k?token=GHSAT0AAAAAACMFWG5Q2E6VRWUSVDKI2VQEZMU2IMQ')
df.head()

,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp
0,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,900,PEANUTS,STIR-FRY,VEGAN,UMAMI,LUNCH,100000
1,Biryani,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""saffron"", ""milk"", ""hot green chili peppers""...",1110.7,100,MILK,BOIL,ANYTHING,SALTY,BREAKFAST,40000
2,Best Lemonade,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,700,CRUSTACEANS,STEAM,MEDITERRANEAN,SALTY,LUNCH,20000
3,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,700,CELERY,POACH,KETO,SWEET,BREAKFAST,100000
4,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,1500,GLUTEN,STEAM,VEGAN,TART,DINNER,40000


In [69]:
def metadata_func(row):
  return f"{row['description']},{row['nutrition']}, {row['tags']}, {row['ingredients']}, {row['cooking_methods']}, {row['taste']}"

df['metadata'] = df.apply(metadata_func, axis=1)
df.head()

,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp,metadata
0,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,900,PEANUTS,STIR-FRY,VEGAN,UMAMI,LUNCH,100000,Make and share this Low-Fat Berry Blue Frozen ...
1,Biryani,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""saffron"", ""milk"", ""hot green chili peppers""...",1110.7,100,MILK,BOIL,ANYTHING,SALTY,BREAKFAST,40000,Make and share this Biryani recipe from Food.c...
2,Best Lemonade,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,700,CRUSTACEANS,STEAM,MEDITERRANEAN,SALTY,LUNCH,20000,This is from one of my first Good House Keepi...
3,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,700,CELERY,POACH,KETO,SWEET,BREAKFAST,100000,This dish is best prepared a day in advance to...
4,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,1500,GLUTEN,STEAM,VEGAN,TART,DINNER,40000,Make and share this Cabbage Soup recipe from F...


In [70]:
df = df.iloc[0:10000, :]
df.to_csv('dataframe_metadata_10k.txt', index=False)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_distances

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
class RecommenderSystem:
  def __init__(self, data, content_col):
    self.df = data
    self.content_col = content_col
    self.encoder = None
    self.bank = None

  def fit(self):
    self.encoder = CountVectorizer(stop_words="english", tokenizer=word_tokenize)
    self.bank = self.encoder.fit_transform(self.df[self.content_col])

  def recommend(self, idx, topk=5):
    content = df.loc[idx, self.content_col]
    code = self.encoder.transform([content])
    dist = cosine_distances(code, self.bank)
    rec_idx = dist.argsort()[0, 1:(topk+1)]
    return self.df.loc[rec_idx]

In [37]:
recsys = RecommenderSystem(data=df, content_col="metadata")
recsys.fit()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [38]:
df.head(5)

,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp,metadata
0,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,900,PEANUTS,STIR-FRY,VEGAN,UMAMI,LUNCH,100000,Make and share this Low-Fat Berry Blue Frozen ...
1,Biryani,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""saffron"", ""milk"", ""hot green chili peppers""...",1110.7,100,MILK,BOIL,ANYTHING,SALTY,BREAKFAST,40000,Make and share this Biryani recipe from Food.c...
2,Best Lemonade,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,700,CRUSTACEANS,STEAM,MEDITERRANEAN,SALTY,LUNCH,20000,This is from one of my first Good House Keepi...
3,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,700,CELERY,POACH,KETO,SWEET,BREAKFAST,100000,This dish is best prepared a day in advance to...
4,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,1500,GLUTEN,STEAM,VEGAN,TART,DINNER,40000,Make and share this Cabbage Soup recipe from F...


In [39]:
recsys.recommend(4)

,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp,metadata
44715,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,character(0),Vegetable,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""cabbage"", ""celery"", ""Equal sugar substitute...",184.0,900,MILK,ROAST,ANYTHING,PLAIN,DINNER,100000,Make and share this Cabbage Soup recipe from F...
63071,Oriental Cabbage,Make and share this Oriental Cabbage recipe fr...,"c(""https://img.sndimg.com/food/image/upload/w_...",Vegetable,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""cabbage"", ""celery ribs"", ""green pepper"", ""o...",123.2,300,CELERY,STIR-FRY,MEDITERRANEAN,PLAIN,BREAKFAST,60000,Make and share this Oriental Cabbage recipe fr...
30980,Hearty Cabbage Soup,Make and share this Hearty Cabbage Soup recipe...,"c(""https://img.sndimg.com/food/image/upload/w_...",Potato,"c(""Vegetable"", ""European"", ""Low Protein"", ""Low...","c(""butter"", ""onion"", ""celery"", ""cabbage"", ""pot...",398.6,900,EGGS,BOIL,ANYTHING,SOUR,SNACK,60000,Make and share this Hearty Cabbage Soup recipe...
43034,Vegetable Dish,Make and share this Vegetable Dish recipe from...,character(0),Vegetable,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""cabbage"", ""green pepper"", ""celery"", ""onion""...",76.6,500,MUSTARD,SAUTE,MEDITERRANEAN,SOUR,ANYTIMES,100000,Make and share this Vegetable Dish recipe from...
18136,Hungarian Vegetarian Cabbage Soup,Make and share this Hungarian Vegetarian Cabba...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Hungarian"", ""European"", ""Low Protein"", ""Low...","c(""tomato juice"", ""green cabbage"", ""carrots"", ...",54.1,500,LUPIN,GRILL,ANYTHING,BITTER,LUNCH,60000,Make and share this Hungarian Vegetarian Cabba...


#Filtering based user profile

In [8]:
#height_cm = 175
#weight_kg = 78
#age = 20
#gender = 'MALE'
#goal = 'WEIGHT_LOSS'
meal_per_day = 3
primary_diet = 'ANYTHING'
#activity_level = 'SEDENTARY'
allergens_user = 'MILK'
#bmi = 25.47
kcal_per_day = 1635
#weight_category = 'Overweight'

In [10]:
extracted_data = df[df['calories_cal'] < kcal_per_day / meal_per_day]
extracted_data.head()

,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp,metadata
0,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,900,PEANUTS,STIR-FRY,VEGAN,UMAMI,LUNCH,100000,Make and share this Low-Fat Berry Blue Frozen ...
2,Best Lemonade,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,700,CRUSTACEANS,STEAM,MEDITERRANEAN,SALTY,LUNCH,20000,This is from one of my first Good House Keepi...
3,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,700,CELERY,POACH,KETO,SWEET,BREAKFAST,100000,This dish is best prepared a day in advance to...
4,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,1500,GLUTEN,STEAM,VEGAN,TART,DINNER,40000,Make and share this Cabbage Soup recipe from F...
5,Best Blackbottom Pie,Make and share this Best Blackbottom Pie recip...,character(0),Pie,"c(""Dessert"", ""Weeknight"", ""Stove Top"", ""< 4 Ho...","c(""graham cracker crumbs"", ""sugar"", ""butter"", ...",437.9,1500,SOYA,STEAM,KETO,SWEET,ANYTIMES,100000,Make and share this Best Blackbottom Pie recip...


In [11]:
def user_filtering(data, primary_diet, allergens_user, meal_times):
    data = data[data['diet_type'].str.strip() == primary_diet]
    data = data[data['allergens'].str.strip() != allergens_user]
    data = data[data['meal_times'].str.strip().isin([meal_times, 'ANYTIMES'])]
    return data

user_filtering(extracted_data, primary_diet, allergens_user, meal_times='BREAKFAST')


,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp,metadata
8,A Jad - Cucumber Pickle,Make and share this A Jad - Cucumber Pickle re...,character(0),Vegetable,"c(""Thai"", ""Asian"", ""Free Of..."", ""< 30 Mins"")","c(""rice vinegar"", ""haeo"")",4.3,1500,LUPIN,BOIL,ANYTHING,SWEET,ANYTIMES,100000,Make and share this A Jad - Cucumber Pickle re...
30,Chicha Peruana,Chicha (corn beer). Chicha is made in South an...,character(0),Beverages,"c(""Mexican"", ""Free Of..."", ""Weeknight"", ""Easy"")","c(""piloncillo cone"", ""Nottingham dry yeast"")",21.1,300,EGGS,GRILL,ANYTHING,SALTY,BREAKFAST,60000,Chicha (corn beer). Chicha is made in South an...
41,All Purpose Quick Mix With 28 Variations,--Adopted Recipe--\nDon't be intimidated by al...,character(0),High In...,"c(""< 60 Mins"", ""Easy"")","c(""whole wheat flour"", ""flour"", ""baking powder...",529.3,100,CRUSTACEANS,STEAM,ANYTHING,SWEET,ANYTIMES,100000,--Adopted Recipe--\nDon't be intimidated by al...
56,"Capsicum, Tomato and Crunchy Bread Salad",I've made this one with stale toast bread it w...,character(0),Low Protein,"c(""Low Cholesterol"", ""Weeknight"", ""< 4 Hours"")","c(""garlic"", ""egg tomatoes"", ""olive oil"", ""oliv...",283.9,700,CRUSTACEANS,GRILL,ANYTHING,SAVORY,BREAKFAST,60000,I've made this one with stale toast bread it w...
57,Chaat Masala,"Asafoetida, mango powder and black salt distin...",character(0),Asian,"c(""Indian"", ""< 30 Mins"", ""Easy"")","c(""cumin seeds"", ""fennel seeds"", ""garam masala...",125.3,1300,FISH,ROAST,ANYTHING,UMAMI,BREAKFAST,20000,"Asafoetida, mango powder and black salt distin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99936,Wheat and Milk Free Brownies,This is a recipe for those wheat or lactose in...,character(0),Bar Cookie,"c(""Dessert"", ""Lunch/Snacks"", ""Cookie & Brownie...","c(""butter"", ""light brown sugar"", ""eggs"", ""vani...",106.9,1500,SOYA,SIMMER,ANYTHING,SALTY,ANYTIMES,40000,This is a recipe for those wheat or lactose in...
99953,Awesome Almond Bark,Make and share this Awesome Almond Bark recipe...,character(0),Candy,"c(""Dessert"", ""Kid Friendly"", ""< 30 Mins"", ""Beg...","""white chocolate pieces""",281.8,900,SESAME,GRILL,ANYTHING,MILD,BREAKFAST,60000,Make and share this Awesome Almond Bark recipe...
99961,Bertha's &quot;highly-Requested&quot; Macaroni...,This is my mother-in-law's recipe for mac-n-ch...,character(0),Cheese,"""< 60 Mins""","c(""pasta"", ""colby-monterey jack cheese"", ""butt...",422.1,500,MUSTARD,ROAST,ANYTHING,SAVORY,ANYTIMES,20000,This is my mother-in-law's recipe for mac-n-ch...
99973,Thai Steamed Halibut,This recipe has a very distinct taste from the...,character(0),Halibut,"c(""Thai"", ""Asian"", ""< 60 Mins"", ""Steam"")","c(""low-fat coconut milk"", ""lemon, rind of"", ""g...",225.5,900,NUTS,POACH,ANYTHING,SPICY,ANYTIMES,80000,This recipe has a very distinct taste from the...


#Finishing

##Import Library

In [71]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_distances

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##Function

In [72]:
class RecommenderSystem:
    def __init__(self, data, content_col):
        self.df = pd.read_csv(data)
        self.content_col = content_col
        self.encoder = None
        self.bank = None

    def fit(self):
        self.encoder = CountVectorizer(stop_words="english", tokenizer=word_tokenize)
        self.bank = self.encoder.fit_transform(self.df[self.content_col])

    def log_filtering(self, idx, topk=5):
        content = self.df.loc[idx, self.content_col]
        code = self.encoder.transform([content])
        dist = cosine_distances(code, self.bank)
        rec_idx = dist.argsort()[0, 1:(topk+1)]
        return self.df.loc[rec_idx]

    def datauser_filtering(self, primary_diet, allergens_user, meal_times, topk=5):
        df_copy = self.df.copy()
        df_copy = df_copy[df_copy['diet_type'].str.strip() == primary_diet]
        df_copy = df_copy[df_copy['allergens'].str.strip() != allergens_user]
        df_copy = df_copy[df_copy['meal_times'].str.strip().isin([meal_times, 'ANYTIMES'])]

        rec_idx = df_copy.index[:topk]
        return self.df.loc[rec_idx]

    def content_based_filtering(self, primary_diet, allergens_user, meal_times, idx=None, topk=5):
        if idx:
            # Jika idx adalah index makanan pada dataframe yang terakhir kali dipesan
            self.log_filtering(idx, topk)
            return self.datauser_filtering(primary_diet, allergens_user, meal_times, topk)
        else:
            # Jika idx tidak diberikan, jalankan datauser_filtering
            return self.datauser_filtering(primary_diet, allergens_user, meal_times, topk)

##Training Data

In [73]:
dataset = 'https://raw.githubusercontent.com/althafnafi/incubate-app/main/dataset/dataset_makanan10k_metadata?token=GHSAT0AAAAAACMFWG5RO7XWC67GDB4AUR6OZMU35SA'
recsys = RecommenderSystem(data=dataset, content_col='metadata')
recsys.fit()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


##Test Dataset

In [76]:
df = pd.read_csv(dataset)
df.head()

,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp,metadata
0,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,900,PEANUTS,STIR-FRY,VEGAN,UMAMI,LUNCH,100000,Make and share this Low-Fat Berry Blue Frozen ...
1,Biryani,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""saffron"", ""milk"", ""hot green chili peppers""...",1110.7,100,MILK,BOIL,ANYTHING,SALTY,BREAKFAST,40000,Make and share this Biryani recipe from Food.c...
2,Best Lemonade,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,700,CRUSTACEANS,STEAM,MEDITERRANEAN,SALTY,LUNCH,20000,This is from one of my first Good House Keepi...
3,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,700,CELERY,POACH,KETO,SWEET,BREAKFAST,100000,This dish is best prepared a day in advance to...
4,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,1500,GLUTEN,STEAM,VEGAN,TART,DINNER,40000,Make and share this Cabbage Soup recipe from F...


##Test Content Based Filtering

In [77]:
recsys.content_based_filtering(primary_diet = 'VEGAN', allergens_user = 'GLUTEN', meal_times = 'BREAKFAST', idx=4)

,food_name,description,image,nutrition,tags,ingredients,calories_cal,weight_gr,allergens,cooking_methods,diet_type,taste,meal_times,price_rp,metadata
53,Blue Stuffed Mushrooms,Make and share this Blue Stuffed Mushrooms rec...,character(0),Free Of...,"c(""< 15 Mins"", ""Easy"")","c(""blue cheese"", ""cream cheese"", ""half-and-hal...",246.3,100,CELERY,BOIL,VEGAN,MILD,BREAKFAST,100000,Make and share this Blue Stuffed Mushrooms rec...
97,Cheesy Chicken Noodle Casserole,Make and share this Cheesy Chicken Noodle Cass...,character(0),Chicken Breast,"c(""Chicken"", ""Poultry"", ""Meat"", ""< 60 Mins"", ""...","c(""chicken breasts"", ""olive oil"", ""Velveeta ch...",544.0,500,SOYA,SIMMER,VEGAN,SALTY,BREAKFAST,60000,Make and share this Cheesy Chicken Noodle Cass...
99,&quot;21&quot; Apple Pie,Make and share this &quot;21&quot; Apple Pie r...,character(0),Pie,"c(""Dessert"", ""Apple"", ""Fruit"", ""Low Protein"", ...","c(""all-purpose flour"", ""salt"", ""unsalted butte...",694.3,1100,SULPHITES,GRILL,VEGAN,SPICY,ANYTIMES,80000,Make and share this &quot;21&quot; Apple Pie r...
112,Chicken Curry,Accompany curry with pappadams and chutney. ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Curries,"c(""Chicken"", ""Poultry"", ""Meat"", ""Asian"", ""Indi...","c(""cumin seeds"", ""red pepper flakes"", ""turmeri...",360.1,300,FISH,BAKE,VEGAN,UMAMI,BREAKFAST,60000,Accompany curry with pappadams and chutney. ...
116,Chicken Liver Pate,Make and share this Chicken Liver Pate recipe ...,character(0),Chicken Livers,"c(""Chicken"", ""Beef Organ Meats"", ""Beef Liver"",...","c(""butter"", ""cream cheese"", ""chicken liver"", ""...",201.2,300,MILK,SIMMER,VEGAN,SAVORY,ANYTIMES,60000,Make and share this Chicken Liver Pate recipe ...
